# Chapter Summarizer Inference
Create a serverless endoint using a pre-baked docker container.

In [2]:
!pip install sagemaker botocore boto3 awscli --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.7/660.7 kB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 42.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 15.8 MB/s eta 0:00:00:00:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.129.0-py2.py3-none-any.whl size=897208 sha256=f4eb85012494a4a95e9ad9cf418920911cc3317437af322d0d14779208a881d2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/84/3a/b2/319c1a71b30ae2104794df12e28c53426110a2bb9c973f9a5d
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.44
    Uninstalling botocore-1.29.44:
      Successfully uninstalled botocore-1.29.44
  Attempting uninstall: boto3
    Found existing installat

In [3]:
BASE_NAME = "chpater-sum-gpt3"
SERVE_IMG_URI = "957269117416.dkr.ecr.eu-central-1.amazonaws.com/chapter-sum-gpt3:latest"

## Define role, model name, and endpoint name

In [4]:
import boto3
import sagemaker
from sagemaker import Session
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

sagemaker_role = sagemaker.get_execution_role()

model_name = f"{BASE_NAME}-model"
endpoint_name = f"{BASE_NAME}-endpoint"
print(f"Model name: {model_name}")
print(f"Endpoint name: {endpoint_name}")
print(f"Image URI: {SERVE_IMG_URI}")
print(f"Role: {sagemaker_role}")

Model name: chpater-sum-gpt3-model
Endpoint name: chpater-sum-gpt3-endpoint
Image URI: 957269117416.dkr.ecr.eu-central-1.amazonaws.com/chapter-sum-gpt3:latest
Role: arn:aws:iam::957269117416:role/service-role/AmazonSageMaker-ExecutionRole-20230121T194089


## Create a SageMaker model, serverless config, and serverless endpoint

In [6]:
from sagemaker.serverless import ServerlessInferenceConfig

#create a SM model using a pre-baked docker container
summarizer_model = Model(
    name=model_name,
    role=sagemaker_role,
    image_uri=SERVE_IMG_URI,
    predictor_cls=Predictor,
)

# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb=2048,
  max_concurrency=2,
)

# create an endpoint config and endpoint.
serverless_predictor = summarizer_model.deploy(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    serverless_inference_config=serverless_config
)

print(f"\nCreated model: {summarizer_model.name}")
print(f"Created endpoint: {serverless_predictor.endpoint_name}")

-----!
Created model: chpater-sum-gpt3-model
Created endpoint: chpater-sum-gpt3-endpoint


## Getting prediction from an endpoint

In [10]:
import json

sm_runtime= boto3.client('runtime.sagemaker')
input_path = "/home/ec2-user/SageMaker/chapter_summarization_api/src/summarizer/resources/chapter/01.txt"

with open(input_path, "r") as fp:
    long_text = fp.read()
    
    payload_json = json.dumps({'text': long_text})

    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name, 
        Body=payload_json.encode('utf-8'), 
        ContentType="application/json",
        Accept="application/json"
    )

    response_body = response['Body'].read().decode('utf-8')
    summary_text = json.loads(response_body)["summary"]
    print(f"Summary:\n{summary_text}")

Summary:
: Alice follows a White Rabbit into a rabbit-hole, and falls down a deep well. She is eventually rescued by a mouse, sees a beautiful garden, and finally escapes from the hall by unlocking a door with a key she finds on a small glass table. 

Alice found a key and a bottle of wine on her way to the little door, but she got lost trying to find the right way to shut it. She ate a cake to make herself grow, but didn't grow at all.


## Clean up Resources

In [5]:
sm = boto3.client("sagemaker")

response = sm.delete_endpoint(EndpointName=endpoint_name)
print(f"Delete endpoint response: {response}")

response = sm.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Delete endpoint configuration response: {response}")

response = sm.delete_model(ModelName=model_name)
print(f"Delete model response: {response}")

Delete endpoint response: {'ResponseMetadata': {'RequestId': '728ebb45-350c-478d-aeee-1a7b539fb093', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '728ebb45-350c-478d-aeee-1a7b539fb093', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Wed, 25 Jan 2023 02:35:16 GMT'}, 'RetryAttempts': 0}}
Delete endpoint configuration response: {'ResponseMetadata': {'RequestId': 'b7c2a294-579c-49f9-ae37-bc2ec2acea55', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b7c2a294-579c-49f9-ae37-bc2ec2acea55', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Wed, 25 Jan 2023 02:35:16 GMT'}, 'RetryAttempts': 0}}
Delete model response: {'ResponseMetadata': {'RequestId': '04ce7c05-3358-413b-839b-10bd04e74f54', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '04ce7c05-3358-413b-839b-10bd04e74f54', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Wed, 25 Jan 2023 02:35:16 GMT'}, 'RetryAttempts

In [1]:
!python --version

Python 3.8.13
